In [1]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from geopy.adapters import AioHTTPAdapter
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
#%load_ext autotime
import geopandas as gpd
import geopy
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm.notebook import tqdm_notebook
import reverse_geocoder as rg


In [2]:
mcdw = pd.read_csv('../data/mcdw.20230530.csv')
mcdw.info()

C:\Users\Aaron\AppData\Local\Temp\ipykernel_2012\2137799343.py:1: DtypeWarning: Columns (9,13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  mcdw = pd.read_csv('../data/mcdw.20230530.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907855 entries, 0 to 907854
Data columns (total 36 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   WMO ID                                             907855 non-null  int64  
 1   YEAR-MONTH                                         907855 non-null  object 
 2   LONGITUDE (deg)                                    907855 non-null  float64
 3   LATITUDE (deg)                                     907855 non-null  float64
 4   ELEVATION (m)                                      907855 non-null  float64
 5   STATION PRESSURE (0.1 mb)                          907855 non-null  int64  
 6   STATION PRESSURE SOURCE FLAG                       826655 non-null  object 
 7   MEAN SEA LEVEL PRESSURE (0.1 mb)                   825674 non-null  float64
 8   MEAN SEA LEVEL PRESSURE SOURCE FLAG                907855 non-null  object

In [3]:
#Droppping columns not relavent 
mcdw.drop(['ELEVATION (m)', 'STATION PRESSURE (0.1 mb)','STATION PRESSURE SOURCE FLAG', 'MEAN SEA LEVEL PRESSURE (0.1 mb)', 'MEAN SEA LEVEL PRESSURE SOURCE FLAG', 'MEAN SEA LEVEL PRESSURE HEIGHT FLAG', 'VAPOR PRESSURE (0.1 mb)', 'VAPOR PRESSURE SOURCE FLAG', 'VAPOR PRESSURE DEPARTURE (0.1 mb)', 'VAPOR PRESSURE DEPARTURE SOURCE FLAG'], axis=1, inplace=True)

In [4]:
#removing data with unknow coordinates
#mcdw = mcdw[(mcdw['LATITUDE (deg)'] != -9999.0000) & (mcdw['LONGITUDE (deg)'] != -9999.0000)]

In [5]:
len(mcdw['LATITUDE (deg)'])

907855

In [6]:
len(mcdw['LONGITUDE (deg)'])

907855

In [7]:
#trying to create pair of values inside the df
#mcdw['lat_long'] = mcdw['LATITUDE (deg)'] & mcdw['LONGITUDE (deg)']
#mcdw['lat_long']=mcdw.apply(lambda x:'%s,%s' % (x['LATITUDE (deg)'],x['LONGITUDE (deg)']),axis=1)
#mcdw['Coordinates'] = mcdw.apply(lambda row: (row['LATITUDE (deg)'], row['LONGITUDE (deg)']), axis=1)
#mcdw['lat_long'] = mcdw[mcdw['lat_long'].astype(str)]
#mcdw['geom'] = mcdw['LATITUDE (deg)'].map(str) + '','' + mcdw['LONGITUDE (deg)'].map(str)
mcdw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907855 entries, 0 to 907854
Data columns (total 26 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   WMO ID                                             907855 non-null  int64  
 1   YEAR-MONTH                                         907855 non-null  object 
 2   LONGITUDE (deg)                                    907855 non-null  float64
 3   LATITUDE (deg)                                     907855 non-null  float64
 4   MINIMUM TEMPERATURE (deg C)                        907855 non-null  int64  
 5   MINIMUM TEMPERATURE SOURCE FLAG                    605950 non-null  object 
 6   MEAN TEMPERATURE (deg C)                           907855 non-null  int64  
 7   MEAN TEMPERATURE SOURCE FLAG                       892136 non-null  object 
 8   MEAN TEMPERATURE DEPARTURE (deg C)                 907855 non-null  int64 

In [8]:
#mcdw['lat_long'] = mcdw['lat_long'].astype(str)

In [9]:
#other option of creating tuple list for the geolocator
lat = mcdw['LATITUDE (deg)']
lon = mcdw['LONGITUDE (deg)']

result = []
for i in range(len(lat)):
     #Adding to result based on indexes
     result.append((lat[i], lon[i]))

In [10]:
result

[(-9999.0, -9999.0),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8

In [11]:
#Need to group into regions These are lots of ways I did not get to work
#lat = mcdw['LATITUDE (deg)']
#lang = mcdw['LONGITUDE (deg)']
#location = geolocator.reverse("{}, {}".format(mcdw['LATITUDE (deg)'], mcdw['LONGITUDE (deg)']))

#geolocator = Nominatim(user_agent="google")
#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#location = geolocator.reverse(mcdw['lat_long'])
 #df['location'] = df['name'].apply(geocode)
#mcdw["location"] = mcdw['lat_long'].apply(lambda el: geolocator.reverse(el).raw["address"])
#mcdw["city"] = list(map(lambda el: geolocator.reverse(el).raw["address"], mcdw["lat_long"]))
#mcdw["city"] = list(map(lambda el: geolocator.reverse(el).raw["address"], result))

#mcdw['location'] = geolocator.reverse(f'{lat}, {lang}')

In [12]:
#locator = Nominatim(user_agent='google', timeout=10)
#rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

In [13]:
#type(result[0])
#location = locator.reverse(result[0])
#location.raw

In [14]:
#So that made a nice little dic of locations... 

location_dic = rg.search(result)

Loading formatted geocoded file...


In [15]:
len(location_dic)

907855

In [16]:
#turning dictionary into df so as to merge back in with the mcddw df
location_df = pd.DataFrame(location_dic)
location_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907855 entries, 0 to 907854
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   lat     907855 non-null  object
 1   lon     907855 non-null  object
 2   name    907855 non-null  object
 3   admin1  907855 non-null  object
 4   admin2  907855 non-null  object
 5   cc      907855 non-null  object
dtypes: object(6)
memory usage: 41.6+ MB


In [17]:
#Formatting for merge
location_df['lat'] = location_df['lat'].astype(float)
location_df['lon'] = location_df['lon'].astype(float)

In [55]:
#testing country and state name.
US = location_df[location_df['cc'] == 'US']
US

,lat,lon,name,admin1,admin2,cc
541786,71.29,-156.79,Barrow,Alaska,North Slope Borough,US
541787,71.29,-156.79,Barrow,Alaska,North Slope Borough,US
541788,71.29,-156.79,Barrow,Alaska,North Slope Borough,US
541789,71.29,-156.79,Barrow,Alaska,North Slope Borough,US
541790,71.29,-156.79,Barrow,Alaska,North Slope Borough,US
...,...,...,...,...,...,...
828565,19.73,-155.09,Hilo,Hawaii,Hawaii County,US
828566,19.73,-155.09,Hilo,Hawaii,Hawaii County,US
828567,19.73,-155.09,Hilo,Hawaii,Hawaii County,US
828568,19.73,-155.09,Hilo,Hawaii,Hawaii County,US


In [60]:
#Rounding lat and lon numbers to hopefully make merging better
location_df= location_df.round(decimals=5)
mcdw['LONGITUDE (deg)'] = mcdw['LONGITUDE (deg)'].round(decimals = 5)
mcdw['LONGITUDE (deg)'] = mcdw['LONGITUDE (deg)'].round(decimals = 5)
US = US.round(decimals = 5)

In [62]:
mcdw['lat'] = mcdw['LATITUDE (deg)']
mcdw['lon'] = mcdw['LONGITUDE (deg)']
us_weather = mcdw.merge(location_df, how = 'inner', on = ['lat', 'lon'])
us_weather

,WMO ID,YEAR-MONTH,LONGITUDE (deg),LATITUDE (deg),MINIMUM TEMPERATURE (deg C),MINIMUM TEMPERATURE SOURCE FLAG,MEAN TEMPERATURE (deg C),MEAN TEMPERATURE SOURCE FLAG,MEAN TEMPERATURE DEPARTURE (deg C),MEAN TEMPERATURE DEPARTURE SOURCE FLAG,...,SUNSHINE DURATION AS LONG TERM NORMAL (%),SUNSHINE DURATION AS LONG TERM NORMAL SOURCE FLAG,VERSION,STATION NAME,lat,lon,name,admin1,admin2,cc
0,8495,1987-01,-5.35,36.15,-9999,NaN,131,L,-9999,NaN,...,61,L,mcdw.20230530,GIBRALTAR (8495-1),36.15,-5.35,Gibraltar,,,GI
1,8495,1987-01,-5.35,36.15,-9999,NaN,131,L,-9999,NaN,...,61,L,mcdw.20230530,GIBRALTAR (8495-1),36.15,-5.35,Gibraltar,,,GI
2,8495,1987-01,-5.35,36.15,-9999,NaN,131,L,-9999,NaN,...,61,L,mcdw.20230530,GIBRALTAR (8495-1),36.15,-5.35,Gibraltar,,,GI
3,8495,1987-01,-5.35,36.15,-9999,NaN,131,L,-9999,NaN,...,61,L,mcdw.20230530,GIBRALTAR (8495-1),36.15,-5.35,Gibraltar,,,GI
4,8495,1987-01,-5.35,36.15,-9999,NaN,131,L,-9999,NaN,...,61,L,mcdw.20230530,GIBRALTAR (8495-1),36.15,-5.35,Gibraltar,,,GI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2610947,97230,2023-04,115.18,-8.75,249,T,278,T,-9999,NaN,...,93,T,mcdw.20230530,DENPASAR/NGURAH RAI,-8.75,115.18,Kelanabian,Bali,,ID
2610948,97230,2023-04,115.18,-8.75,249,T,278,T,-9999,NaN,...,93,T,mcdw.20230530,DENPASAR/NGURAH RAI,-8.75,115.18,Kelanabian,Bali,,ID
2610949,97230,2023-04,115.18,-8.75,249,T,278,T,-9999,NaN,...,93,T,mcdw.20230530,DENPASAR/NGURAH RAI,-8.75,115.18,Kelanabian,Bali,,ID
2610950,97230,2023-04,115.18,-8.75,249,T,278,T,-9999,NaN,...,93,T,mcdw.20230530,DENPASAR/NGURAH RAI,-8.75,115.18,Kelanabian,Bali,,ID


In [64]:
us_weather = us_weather[us_weather['cc'] == 'US']
us_weather

,WMO ID,YEAR-MONTH,LONGITUDE (deg),LATITUDE (deg),MINIMUM TEMPERATURE (deg C),MINIMUM TEMPERATURE SOURCE FLAG,MEAN TEMPERATURE (deg C),MEAN TEMPERATURE SOURCE FLAG,MEAN TEMPERATURE DEPARTURE (deg C),MEAN TEMPERATURE DEPARTURE SOURCE FLAG,...,SUNSHINE DURATION AS LONG TERM NORMAL (%),SUNSHINE DURATION AS LONG TERM NORMAL SOURCE FLAG,VERSION,STATION NAME,lat,lon,name,admin1,admin2,cc
1614543,72242,1987-01,-94.80,29.30,-9999,NaN,117,L,2,L,...,-9999,NaN,mcdw.20230530,"GALVESTON, TX.",29.30,-94.80,Galveston,Texas,Galveston County,US
1614544,72242,1987-01,-94.80,29.30,-9999,NaN,117,L,2,L,...,-9999,NaN,mcdw.20230530,"GALVESTON, TX.",29.30,-94.80,Galveston,Texas,Galveston County,US
1614545,72242,1987-01,-94.80,29.30,-9999,NaN,117,L,2,L,...,-9999,NaN,mcdw.20230530,"GALVESTON, TX.",29.30,-94.80,Galveston,Texas,Galveston County,US
1614546,72242,1987-01,-94.80,29.30,-9999,NaN,117,L,2,L,...,-9999,NaN,mcdw.20230530,"GALVESTON, TX.",29.30,-94.80,Galveston,Texas,Galveston County,US
1614547,72242,1987-01,-94.80,29.30,-9999,NaN,117,L,2,L,...,-9999,NaN,mcdw.20230530,"GALVESTON, TX.",29.30,-94.80,Galveston,Texas,Galveston County,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016852,72514,2023-04,-76.92,41.25,57,T,126,T,31,T,...,-9999,NaN,mcdw.20230530,"WILLIAMSPORT/LYCOMING COUNTY,",41.25,-76.92,Montoursville,Pennsylvania,Lycoming County,US
2016853,72514,2023-04,-76.92,41.25,57,T,126,T,31,T,...,-9999,NaN,mcdw.20230530,"WILLIAMSPORT/LYCOMING COUNTY,",41.25,-76.92,Montoursville,Pennsylvania,Lycoming County,US
2016854,72514,2023-04,-76.92,41.25,57,T,126,T,31,T,...,-9999,NaN,mcdw.20230530,"WILLIAMSPORT/LYCOMING COUNTY,",41.25,-76.92,Montoursville,Pennsylvania,Lycoming County,US
2016855,72514,2023-04,-76.92,41.25,57,T,126,T,31,T,...,-9999,NaN,mcdw.20230530,"WILLIAMSPORT/LYCOMING COUNTY,",41.25,-76.92,Montoursville,Pennsylvania,Lycoming County,US


In [52]:
# Create a dictionary mapping states to regions
region_mapping = {
    'Tennessee': 'SOUTH EASTERN',
    'NC': 'SOUTH EASTERN',
    'SC': 'SOUTH EASTERN',
    'GA': 'SOUTH EASTERN',
    'AL': 'SOUTH EASTERN',
    'MS': 'SOUTH EASTERN',
    'FL': 'SOUTH EASTERN',
    'KY': 'NORTH EASTERN',
    'VA': 'NORTH EASTERN',
    'WV': 'NORTH EASTERN',
    'IN': 'NORTH EASTERN',
    'OH': 'NORTH EASTERN',
    'MI': 'NORTH EASTERN',
    'PA': 'NORTH EASTERN',
    'DC': 'NORTH EASTERN',
    'DE': 'NORTH EASTERN',
    'NJ': 'NORTH EASTERN',
    'NY': 'NORTH EASTERN',
    'CT': 'NORTH EASTERN',
    'MA': 'NORTH EASTERN',
    'VT': 'NORTH EASTERN',
    'NH': 'NORTH EASTERN',
    'ME': 'NORTH EASTERN',
    'RI': 'NORTH EASTERN',
    'ND': 'NORTH CENTRAL',
    'MN': 'NORTH CENTRAL',
    'WI': 'NORTH CENTRAL',
    'SD': 'NORTH CENTRAL',
    'NE': 'NORTH CENTRAL',
    'KS': 'NORTH CENTRAL',
    'MO': 'NORTH CENTRAL',
    'IL': 'NORTH CENTRAL',
    'IA': 'NORTH CENTRAL',
    'OK': 'SOUTH CENTRAL',
    'AR': 'SOUTH CENTRAL',
    'LA': 'SOUTH CENTRAL',
    'TX': 'SOUTH CENTRAL',
    'CA': 'SOUTH WEST',
    'NV': 'SOUTH WEST',
    'UT': 'SOUTH WEST',
    'AZ': 'SOUTH WEST',
    'CO': 'SOUTH WEST',
    'NM': 'SOUTH WEST',
    'WA': 'NORTH WEST',
    'OR': 'NORTH WEST',
    'ID': 'NORTH WEST',
    'MT': 'NORTH WEST',
    'WY': 'NORTH WEST',


}

# Create a 'region' column based on the 'states' column
us_weather['region'] = us_weather['admin1'].map(region_mapping)

In [53]:
us_weather.to_csv('us_weather.csv', index = True)

In [54]:
us_weather.head()

,WMO ID,YEAR-MONTH,LONGITUDE (deg),LATITUDE (deg),MINIMUM TEMPERATURE (deg C),MINIMUM TEMPERATURE SOURCE FLAG,MEAN TEMPERATURE (deg C),MEAN TEMPERATURE SOURCE FLAG,MEAN TEMPERATURE DEPARTURE (deg C),MEAN TEMPERATURE DEPARTURE SOURCE FLAG,...,SUNSHINE DURATION AS LONG TERM NORMAL SOURCE FLAG,VERSION,STATION NAME,lat,lon,name,admin1,admin2,cc,region
0,72242,1987-01,-94.8,29.3,-9999,NaN,117,L,2,L,...,NaN,mcdw.20230530,"GALVESTON, TX.",29.3,-94.8,Galveston,Texas,Galveston County,US,NaN
1,72242,1987-01,-94.8,29.3,-9999,NaN,117,L,2,L,...,NaN,mcdw.20230530,"GALVESTON, TX.",29.3,-94.8,Galveston,Texas,Galveston County,US,NaN
2,72242,1987-01,-94.8,29.3,-9999,NaN,117,L,2,L,...,NaN,mcdw.20230530,"GALVESTON, TX.",29.3,-94.8,Galveston,Texas,Galveston County,US,NaN
3,72242,1987-01,-94.8,29.3,-9999,NaN,117,L,2,L,...,NaN,mcdw.20230530,"GALVESTON, TX.",29.3,-94.8,Galveston,Texas,Galveston County,US,NaN
4,72242,1987-01,-94.8,29.3,-9999,NaN,117,L,2,L,...,NaN,mcdw.20230530,"GALVESTON, TX.",29.3,-94.8,Galveston,Texas,Galveston County,US,NaN
